<a href="https://colab.research.google.com/github/nwabuike-samuel/NLP-with-disaster-tweets/blob/main/NLP_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')
folder_path = "/content/drive/MyDrive/Colab Notebooks/data" 
files = os.listdir(folder_path)

print(files)


train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['train.csv', 'test.csv', 'sample_submission.csv']


In [ ]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
train_df.tail(15)

,id,keyword,location,text,target
7598,10853,NaN,NaN,Father-of-three Lost Control of Car After Over...,1
7599,10854,NaN,NaN,1.3 #Earthquake in 9Km Ssw Of Anza California ...,1
7600,10855,NaN,NaN,Evacuation order lifted for town of Roosevelt:...,1
7601,10859,NaN,NaN,#breaking #LA Refugio oil spill may have been ...,1
7602,10860,NaN,NaN,a siren just went off and it wasn't the Forney...,1
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1
7604,10863,NaN,NaN,#WorldNews Fallen powerlines on G:link tram: U...,1
7605,10864,NaN,NaN,on the flip side I'm at Walmart and there is a...,1
7606,10866,NaN,NaN,Suicide bomber kills 15 in Saudi security site...,1
7607,10867,NaN,NaN,#stormchase Violent Record Breaking EF-5 El Re...,1


In [ ]:
# Data Peeking.
print(f'Shape of the File : {train_df.shape}\n')
print(f'All the Columns of the Dataframe : \n\n{train_df.columns.values}')

Shape of the File : (7613, 5)

All the Columns of the Dataframe : 

['id' 'keyword' 'location' 'text' 'target']


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
train_df.keyword.value_counts()

fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [ ]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [ ]:
train_df.location.value_counts()

USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: location, Length: 3341, dtype: int64

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
# load the data
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/test.csv')

In [8]:
# preprocess the text data
def preprocess_text(text):
    # convert text to lowercase
    text = text.lower()
    # remove URLs
    text = re.sub(r'http\S+', '', text)
    # tokenize the text
    tokens = word_tokenize(text)
    # remove stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    words = [word for word in tokens if word.isalpha() and word not in stop_words]
    # lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(word) for word in words]
    # join the words back into a string
    return ' '.join(lemmas)

train_df['processed_text'] = train_df['text'].apply(preprocess_text)
test_df['processed_text'] = test_df['text'].apply(preprocess_text)

In [9]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_df['processed_text'], train_df['target'], test_size=0.2, random_state=42)

In [10]:
# create a bag-of-words representation of the text data
count_vectorizer = CountVectorizer()
X_train_counts = count_vectorizer.fit_transform(X_train)
X_test_counts = count_vectorizer.transform(X_test)

In [12]:
# transform the bag-of-words representation into a tf-idf representation
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

In [14]:
# train a Naive Bayes classifier on the tf-idf representation of the data
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

MultinomialNB()

In [15]:
# predict the labels of the test data
y_pred = classifier.predict(X_test_tfidf)

In [19]:
# evaluate the performance of the model
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1 score:', f1_score(y_test, y_pred))


Accuracy: 0.7957977675640184
Precision: 0.8164794007490637
Recall: 0.6718027734976888
F1 score: 0.7371090448013525


In [18]:
# predict the labels of the test data and save the results to a file
test_data_counts = count_vectorizer.transform(test_df['processed_text'])
test_data_tfidf = tfidf_transformer.transform(test_data_counts)
test_df['target'] = classifier.predict(test_data_tfidf)
# test_df[['id', 'target']].to_csv('submission.csv', index=False)


import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# load the data
train_df = pd.read_csv('train.csv')

# preprocessing the text data
def preprocess(text):
    # perform cleaning, tokenization, and other necessary preprocessing steps
    return preprocessed_text

train_df['text'] = train_df['text'].apply(preprocess)

# extract features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_df['text'])
y = train_df['target']

# split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train a logistic regression model
clf = LogisticRegression()
clf.fit(X_train, y_train)

# evaluate the model
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

# load the test data
test_df = pd.read_csv('test.csv')
test_df['text'] = test_df['text'].apply(preprocess)

# extract features for test data
X_test = vectorizer.transform(test_df['text'])

# make predictions
predictions = clf.predict(X_test)

# save the predictions to a csv file
submission = pd.DataFrame({'id': test_df['id'], 'target': predictions})
submission.to_csv('submission.csv', index=False)
